# Imports

In [302]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


# Read Data

In [303]:
X_test = pd.read_csv("PEMS_test", sep=";", header=None, nrows=5)
X_train = pd.read_csv("PEMS_train", sep=";", header=None, nrows=5)
y_test = pd.read_csv("PEMS_testlabels", sep=" ", header=None, nrows=5)
y_train = pd.read_csv("PEMS_trainlabels", sep=";", header=None, nrows=5)

In [304]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,953,954,955,956,957,958,959,960,961,962
0,[0.0154 0.0085 0.0099 0.0108 0.01 0.0111 0.009...,0.0054 0.0051 0.0056 0.0045 0.0037 0.0027 0.00...,0.0164 0.0127 0.0172 0.0126 0.0185 0.0077 0.01...,0.0079 0.0062 0.0068 0.0072 0.0048 0.0046 0.00...,0.0059 0.0051 0.0053 0.0058 0.0063 0.0046 0.00...,0.0138 0.0125 0.0116 0.0131 0.0099 0.01 0.0111...,0.0111 0.0099 0.0105 0.0073 0.0061 0.0058 0.00...,0.0156 0.0175 0.0157 0.0146 0.0157 0.0099 0.00...,0.0154 0.0134 0.0112 0.0128 0.0123 0.0103 0.00...,0.008 0.0067 0.0065 0.004 0.0064 0.0061 0.004 ...,...,0.0027 0.001 0.001 0.0014 0.0013 0.0019 0.0008...,0.0142 0.0074 0.0072 0.0081 0.0055 0.0045 0.00...,0.0125 0.0113 0.0113 0.0119 0.0084 0.0127 0.00...,0.0261 0.0228 0.0246 0.0232 0.0225 0.0246 0.02...,0.0182 0.0131 0.0216 0.0311 0.0139 0.0153 0.01...,0.0208 0.0115 0.0137 0.0124 0.0119 0.0113 0.00...,0.0196 0.0156 0.0201 0.0188 0.018 0.0215 0.014...,0.0123 0.0065 0.0098 0.0077 0.0078 0.0102 0.00...,0.015 0.0113 0.0141 0.013 0.0155 0.0153 0.01 0...,0.014 0.0183 0.0112 0.0107 0.0162 0.0212 0.008...
1,[0.0109 0.0053 0.0096 0.0071 0.0082 0.0096 0.0...,0.0105 0.0055 0.0072 0.0046 0.004 0.0023 0.004...,0.0175 0.0176 0.0144 0.0118 0.0132 0.0128 0.00...,0.0082 0.006 0.0058 0.005 0.0058 0.007 0.003 0...,0.0076 0.0071 0.0078 0.0067 0.0053 0.0049 0.00...,0.0165 0.0144 0.0128 0.011 0.0108 0.0102 0.010...,0.0087 0.0076 0.0069 0.0102 0.0102 0.0073 0.00...,0.0148 0.0181 0.0167 0.0139 0.0138 0.0097 0.00...,0.0133 0.0127 0.0154 0.0109 0.0124 0.0072 0.00...,0.0037 0.0055 0.0063 0.0049 0.0067 0.0044 0.00...,...,0.0015 0.0014 0.0012 0.0029 0.0023 0.0013 0.00...,0.0067 0.0072 0.0058 0.0052 0.0042 0.0055 0.00...,0.0159 0.0112 0.0179 0.0103 0.0106 0.0069 0.01...,0.0249 0.0243 0.0232 0.0232 0.023 0.0242 0.023...,0.0096 0.0049 0.0088 0.0054 0.0095 0.0099 0.00...,0.011 0.0137 0.0177 0.0136 0.0119 0.0101 0.010...,0.0189 0.0194 0.0257 0.016 0.0201 0.015 0.0191...,0.0065 0.009 0.0098 0.0082 0.008 0.0061 0.0044...,0.015 0.0141 0.022 0.0119 0.0147 0.0118 0.0147...,0.009 0.0206 0.0141 0.0102 0.0167 0.0123 0.011...
2,[0.0092 0.0088 0.0104 0.0093 0.0092 0.0077 0.0...,0.0082 0.0076 0.0074 0.0053 0.0057 0.0069 0.00...,0.016 0.0171 0.0162 0.0111 0.0094 0.0109 0.011...,0.0054 0.0075 0.006 0.0056 0.0065 0.0068 0.005...,0.0066 0.0102 0.0084 0.0119 0.0076 0.005 0.004...,0.0141 0.024 0.0173 0.0145 0.0113 0.0126 0.010...,0.0091 0.0093 0.0073 0.0068 0.0073 0.0057 0.00...,0.0193 0.0188 0.0183 0.0146 0.0164 0.0117 0.01...,0.0174 0.0175 0.014 0.0175 0.0169 0.0121 0.010...,0.0086 0.0088 0.0076 0.0066 0.0044 0.0072 0.00...,...,0.0034 0.0024 0.0026 0.002 0.002 0.001 0.0004 ...,0.0068 0.0048 0.0069 0.0085 0.0033 0.0051 0.00...,0.0148 0.0181 0.0158 0.0111 0.0128 0.0118 0.00...,0.0238 0.0237 0.0277 0.0242 0.0228 0.0236 0.02...,0.0056 0.0051 0.0051 0.0069 0.0049 0.0043 0.00...,0.0134 0.0115 0.0172 0.0129 0.0085 0.012 0.012...,0.0305 0.0286 0.0207 0.0177 0.0207 0.022 0.017...,0.01 0.0082 0.0083 0.0068 0.0078 0.0075 0.0061...,0.0245 0.0234 0.0171 0.0163 0.0158 0.0166 0.01...,0.0265 0.0257 0.0199 0.0226 0.023 0.0223 0.018...
3,[0.0243 0.0257 0.0255 0.0215 0.0192 0.0221 0.0...,0.0072 0.0051 0.009 0.0038 0.0056 0.0047 0.003...,0.0241 0.0161 0.0164 0.0127 0.0138 0.0135 0.00...,0.007 0.0076 0.0057 0.0066 0.0076 0.0041 0.005...,0.0056 0.0043 0.0081 0.0049 0.0087 0.005 0.008...,0.0154 0.0148 0.011 0.0104 0.0101 0.0139 0.008...,0.0072 0.009 0.007 0.0065 0.0057 0.0054 0.0063...,0.025 0.0198 0.0177 0.0153 0.012 0.0136 0.0105...,0.0408 0.0421 0.0391 0.016 0.0164 0.017 0.0086...,0.0333 0.0315 0.0274 0.0286 0.0261 0.0259 0.02...,...,0.0131 0.0139 0.0156 0.0126 0.0118 0.011 0.010...,0.0171 0.019 0.0169 0.018 0.0207 0.0167 0.0207...,0.0085 0.0064 0.0123 0.0092 0.0072 0.0074 0.00...,0.0304 0.0339 0.0331 0.0317 0.0346 0.0331 0.03...,0.0214 0.0193 0.0222 0.0227 0.0272 0.0228 0.02...,0.0288 0.0299 0.0276 0.0318 0.0317 0.03 0.0269...,0.0158 0.0159 0.0148 0.0151 0.0133 0.0108 0.01...,0.0312 0.0358 0.028 0.0262 0.0343 0.0255 0.032...,0.0158 0.01

In [307]:
def transposeMeasurments(df):
    # Iterate over all columns
    for column in range(963):
        # Split by " "
        lists = []
        # Iterate over all rows from one specific columns and store the values as lists in variable lists
        for i in range(len(df)):
            liste = df[column][i].split(" ")
            if liste[0].startswith("["):
                liste[0] = liste[0].replace("[", "")
            
            if liste[-1].endswith("]"):
                liste[-1] = liste[-1].replace("]", "")

            floats = [float(i) for i in liste]
            lists.append(floats)


        neue_listen = [[] for _ in range(len(lists[0]))]
    
        # Iteriere über die Sublisten
        for sublist in lists:
            for index, element in enumerate(sublist):
                neue_listen[index].append(element)
       

        new = pd.DataFrame(neue_listen)
        new = new.transpose()
        new.columns = [f"Sensor {column} - Messwert {index}" for index in range(1, len(neue_listen) + 1)]
        
        df = pd.concat([df, new], axis=1)

    # Drop old columns with the names 0 until 962
    df.drop(columns=[x for x in range(963)], inplace=True)
    return df

# Transform Data with pipeline

In [57]:
# Create a pipeline with StandardScaler and PCA
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: Standard Scaling
    ('pca', PCA(n_components=2)),   # Step 2: PCA
])

# Fit the pipeline on the training data
pipeline.fit(X_train)

# Transform the training and test data using the pipeline
X_train = pipeline.transform(X_train)
X_test = pipeline.transform(X_test)

ValueError: could not convert string to float: '[0.0154 0.0085 0.0099 0.0108 0.01 0.0111 0.0099 0.0081 0.0099 0.0088 0.008 0.0104 0.0084 0.0106 0.0085 0.0097 0.0078 0.0085 0.0097 0.0093 0.011 0.0133 0.0139 0.0143 0.0131 0.0168 0.0201 0.0198 0.0249 0.0241 0.0373 0.0366 0.0545 0.0688 0.0615 0.0764 0.0755 0.0852 0.0924 0.0857 0.0883 0.085 0.0781 0.0795 0.0678 0.0625 0.0635 0.0639 0.0658 0.0563 0.0642 0.0591 0.0631 0.0626 0.0554 0.062 0.0606 0.0548 0.0647 0.0577 0.0623 0.0604 0.0563 0.0729 0.0659 0.0639 0.0629 0.0639 0.057 0.0646 0.0579 0.0577 0.0537 0.0622 0.0741 0.0535 0.0685 0.0597 0.0612 0.0615 0.0639 0.0597 0.0593 0.0501 0.0551 0.0563 0.0597 0.0656 0.0621 0.06 0.0519 0.0627 0.0671 0.0623 0.0553 0.0584 0.0649 0.0609 0.0574 0.058 0.0583 0.0657 0.0594 0.0579 0.0679 0.065 0.0603 0.059 0.0496 0.0579 0.0495 0.0457 0.0461 0.0422 0.0418 0.0465 0.0427 0.0356 0.0345 0.0337 0.033 0.0335 0.0323 0.0305 0.0286 0.0252 0.027 0.0229 0.026 0.0245 0.0263 0.0235 0.0229 0.0194 0.0152 0.0182 0.0175 0.0151 

In [40]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,137700,137701,137702,137703,137704,137705,137706,137707,137708,137709
0,0.0154,0.0085,0.0099,0.0108,0.0100,0.0111,0.0099,0.0081,0.0099,0.0088,...,0.0258,0.0239,0.0202,0.0147,0.0251,0.0165,0.0202,0.0148,0.0140,0.0194
1,0.0109,0.0053,0.0096,0.0071,0.0082,0.0096,0.0081,0.0062,0.0070,0.0056,...,0.0311,0.0231,0.0192,0.0239,0.0191,0.0150,0.0150,0.0165,0.0203,0.0110
2,0.0092,0.0088,0.0104,0.0093,0.0092,0.0077,0.0088,0.0087,0.0082,0.0084,...,0.0541,0.0487,0.0463,0.0509,0.0527,0.0563,0.0442,0.0383,0.0564,0.0351
3,0.0243,0.0257,0.0255,0.0215,0.0192,0.0221,0.0197,0.0215,0.0222,0.0256,...,0.0389,0.0373,0.0339,0.0304,0.0276,0.0306,0.0260,0.0239,0.0249,0.0190
4,0.0294,0.0295,0.0310,0.0263,0.0278,0.0241,0.0264,0.0267,0.0266,0.0251,...,0.0329,0.0279,0.0244,0.0237,0.0211,0.0199,0.0201,0.0184,0.0188,0.0157
